In [1]:
import nideconv
from risk_experiment.utils.data import Subject



/home/ubuntu/miniconda3/lib/python3.8/site-packages/nilearn/input_data/__init__.py:27: FutureWarning: The import path 'nilearn.input_data' is deprecated in version 0.9. Importing from 'nilearn.input_data' will be possible at least until release 0.13.0. Please import from 'nilearn.maskers' instead.
  warnings.warn(message, FutureWarning)
/home/ubuntu/miniconda3/lib/python3.8/site-packages/nilearn/glm/__init__.py:55: FutureWarning: The nilearn.glm module is experimental. It may change in any future release of Nilearn.
  warn('The nilearn.glm module is experimental. '


In [2]:
pupil = Subject('02', '/data/ds-risk').get_pupil('3t2')
saccades = Subject('02', '/data/ds-risk').get_saccades('3t2')
blinks = Subject('02', '/data/ds-risk').get_blinks('3t2')

In [3]:
blinks.index.set_names('test', -1, inplace=True)

/tmp/ipykernel_5703/3408860632.py:1: FutureWarning: In a future version of pandas all arguments of MultiIndex.set_names except for the argument 'names' will be keyword-only.
  blinks.index.set_names('test', -1, inplace=True)


In [ ]:
onsets = pd.concat((blinks, saccades), keys=['blink', 'saccade'], names=['trial_type'])

In [8]:
grf = nideconv.GroupResponseFitter(pupil['pupil'], onsets=saccades, input_sample_rate=500)

/home/ubuntu/git/nideconv/nideconv/group_analysis.py:87: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  self.response_fitters = pd.Series(index=index)
/home/ubuntu/git/nideconv/nideconv/response_fitter.py:91: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  self.X = pd.concat((regressor.X, self.X), 1)
/home/ubuntu/git/nideconv/nideconv/response_fitter.py:91: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  self.X = pd.concat((regressor.X, self.X), 1)
/home/ubuntu/git/nideconv/nideconv/response_fitter.py:91: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  self.X = pd.concat((regressor.X, self.X), 1)
/home/ubuntu/git/nideconv/nideconv/response_fitter.p

In [9]:
grf.add_event